# Feature Engineering

Some fundamentals of exploratory data analysis and feature engineering using the Titanic data set


In [ ]:
# %pip install --quiet --upgrade pip 
# %pip install numpy --quiet
# %pip install PyArrow --quiet
# %pip install Pandas --quiet
# %pip install scikit-learn --quiet